# Distinguishing different sports - using TensorFlow

There are several higher level wrappers for TensorFlow, last not least, Keras.
The following code just serves to showcase raw TensorFlow - it has not been optimized in any way.

In [1]:
import tensorflow as tf
import numpy as np
np.set_printoptions(precision = 3)

In [2]:
model_name = "cnn_tensorflow_10epochs.ckpt"
model_exists = True

In [3]:
learning_rate = 0.001
num_epochs = 1
batch_size = 10


n_train = 7096
n_test = 1775
steps_per_epoch = n_train // batch_size


In [4]:
input_height = 224
input_width = 224
channels = 3
num_classes = 6
dropout = 0.5

In [5]:
# Placeholders
X = tf.placeholder(tf.float32, [None, input_height, input_width, channels])
Y = tf.placeholder(tf.float32, [None, num_classes])
keep_prob = tf.placeholder(tf.float32) 
phase = tf.placeholder(tf.bool) # for batchnorm

In [6]:
# Variables
# weights: (kernel height, kernel width, input size, output size)
weights = {
    'wconv1': tf.Variable(tf.random_normal([3, 3, 3, 32])),
    'wconv2': tf.Variable(tf.random_normal([3, 3, 32, 32])),
    'wconv3': tf.Variable(tf.random_normal([3, 3, 32, 32])),
    'wconv4': tf.Variable(tf.random_normal([3, 3, 32, 32])),
    'wdense1': tf.Variable(tf.random_normal([56*56*32, 256])),
    'wout': tf.Variable(tf.random_normal([256, num_classes]))
}

# biases (output size)
biases = {
    'bconv1': tf.Variable(tf.random_normal([32])),
    'bconv2': tf.Variable(tf.random_normal([32])),
    'bconv3': tf.Variable(tf.random_normal([32])),
    'bconv4': tf.Variable(tf.random_normal([32])),
    'bdense1': tf.Variable(tf.random_normal([256])),
    'bout': tf.Variable(tf.random_normal([num_classes]))
}


In [7]:
# Create model
def convnet(x, weights, biases, dropout, phase):
    
    # shape=(?, 224, 224, 32)
    conv1 = tf.nn.conv2d(x, weights['wconv1'], strides=[1, 1, 1, 1], padding='SAME')
    
    conv1 = tf.contrib.layers.batch_norm(conv1, center=True, scale=True, is_training=phase)
    conv1 = tf.nn.bias_add(conv1, biases['bconv1'])
    conv1 = tf.nn.relu(conv1)
    
    # shape=(?, 224, 224, 32)
    conv2 = tf.nn.conv2d(conv1, weights['wconv2'], strides=[1, 1, 1, 1], padding='SAME')
    conv2 = tf.contrib.layers.batch_norm(conv2, center=True, scale=True, is_training=phase)
    conv2 = tf.nn.bias_add(conv2, biases['bconv2'])
    conv2 = tf.nn.relu(conv2)
    # shape=(?, 112, 112, 32)
    conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    # shape=(?, 112, 112, 32)
    conv3 = tf.nn.conv2d(conv2, weights['wconv3'], strides=[1, 1, 1, 1], padding='SAME')
    conv3 = tf.contrib.layers.batch_norm(conv3, center=True, scale=True, is_training=phase)
    conv3 = tf.nn.bias_add(conv3, biases['bconv3'])
    conv3 = tf.nn.relu(conv3)
    
    # shape=(?, 112, 112, 32)
    conv4 = tf.nn.conv2d(conv3, weights['wconv4'], strides=[1, 1, 1, 1], padding='SAME')
    conv4 = tf.contrib.layers.batch_norm(conv4, center=True, scale=True, is_training=phase)
    conv4 = tf.nn.bias_add(conv4, biases['bconv4'])
    conv4 = tf.nn.relu(conv4)
    # shape=(?, 56, 56, 32)
    conv4 = tf.nn.max_pool(conv4, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

    # Reshape conv4 output to fit fully connected layer input
    fc1 = tf.reshape(conv4, [-1, weights['wdense1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wdense1']), biases['bdense1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['wout']), biases['bout'])
    return out


In [8]:
# Construct model
logits = convnet(X, weights, biases, keep_prob, phase)
prediction = tf.nn.softmax(logits)


In [9]:
# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

In [10]:
# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


In [11]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = 'data/train'
test_data_dir = 'data/test' 

target_size = (224, 224)

train_datagen = ImageDataGenerator(rescale=1. / 255)
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=target_size,
    batch_size=1,
    class_mode='categorical')

#nxt = train_generator.next()
#nxt[0].shape, nxt[1].shape


Using TensorFlow backend.


Found 7096 images belonging to 6 classes.
Found 1775 images belonging to 6 classes.


In [12]:
saver = tf.train.Saver(var_list=tf.global_variables())

In [13]:
init = tf.global_variables_initializer()


In [14]:
if not model_exists:
    
        
    with tf.Session() as sess:

        sess.run(init)

        for epoch in range(num_epochs):
            
            epoch_loss = 0
            epoch_accuracy = 0
            for step in range(steps_per_epoch):

                batch_x, batch_y = train_generator.next()
                sess.run(train_op, feed_dict={X: batch_x, Y: batch_y, keep_prob: dropout, phase: 1})

                batch_loss, batch_accuracy = sess.run([loss_op, accuracy],
                                                      feed_dict={X: batch_x, 
                                                                 Y: batch_y,
                                                                 keep_prob: dropout,
                                                                 phase: 1})
                if step  % 50 == 0: print ("     batch: {}, accuracy: {}".format(step, batch_accuracy))
                epoch_loss += batch_loss
                epoch_accuracy += batch_accuracy
                
            epoch_loss /= steps_per_epoch
            epoch_accuracy /= steps_per_epoch
            print ("Epoch {}: Loss: {}, accuracy: {}".format(epoch, epoch_loss, epoch_accuracy))

        saver.save(sess, model_name)

   

In [15]:
if model_exists:
    with tf.Session() as sess:
        
        sess.run(init)
        saver = tf.train.import_meta_graph('cnn_tensorflow_10epochs.ckpt.meta')
        saver.restore(sess, tf.train.latest_checkpoint('./'))
        test_accuracy = 0
        
        for step in range(n_test):
            batch_x, batch_y = test_generator.next()
            batch_accuracy = sess.run(accuracy, feed_dict={X: batch_x, 
                                                           Y: batch_y,
                                                           keep_prob: 1.0,
                                                           phase: False})
            test_accuracy += batch_accuracy
        test_accuracy /= n_test
        print("Test accuracy: {}".format(test_accuracy))
        

INFO:tensorflow:Restoring parameters from ./cnn_tensorflow_10epochs.ckpt
Test accuracy: 0.16507042253521126
